In [137]:
import pandas as pd
import numpy as np

from mlgear.utils import show
from mlgear.cv import run_cv_model
from mlgear.models import runRidge
from mlgear.metrics import rmse

In [138]:
polls = pd.read_csv('polls.csv')
show(polls)

        election      state  year    stage dem_candidate  \
0      president    Alabama  2004  general        NoName   
1      president    Alabama  2004  general        NoName   
2      president    Alabama  2004  general        NoName   
3      president    Alabama  2004  general        NoName   
4      president    Alabama  2004  general        NoName   
...          ...        ...   ...      ...           ...   
27114   governor  Louisiana  2019  general       Edwards   
27115   governor  Louisiana  2019  general       Edwards   
27116   governor  Louisiana  2019  general       Edwards   
27117   governor  Louisiana  2019  general       Edwards   
27118   governor  Louisiana  2019  general       Edwards   

                                pollster     N           methodology  \
0                             SurveyUSA    735               Unknown   
1                        Capital Survey    645               Unknown   
2                             SurveyUSA    699               Un

In [139]:
disp = polls[(polls['days_until'] < 90) & (polls['election'] == 'president') & (polls['year'] == 2020)]
disp = disp.groupby(['state'])
disp = disp[['polled_margin', 'actual_margin']].mean().reset_index()
show(disp)

            state  polled_margin  actual_margin
0         Alabama     -18.139388     -25.461653
1          Alaska      -8.147500     -10.061191
2         Arizona       4.312013       0.308710
3        Arkansas     -18.676364     -27.620668
4      California      27.367593      29.163223
..            ...            ...            ...
45       Virginia      13.350536      10.114014
46     Washington      24.371739      19.203323
47  West Virginia     -31.196444     -38.935031
48      Wisconsin       8.369448       0.627100
49        Wyoming     -32.987442     -43.111923

[50 rows x 3 columns]
(50, 3)


In [140]:
disp2 = polls[(polls['days_until'] < 90) & (polls['election'] == 'president') & (polls['year'] == 2016)]
disp2 = disp2.groupby(['state',])
disp2 = disp2[['margin_diff', 'actual_margin']].mean().reset_index()
disp2.columns = ['state', '2016_margin_diff', '2016_actual_margin']
show(disp2)

            state  2016_margin_diff  2016_actual_margin
0         Alabama          8.220000          -27.720000
1          Alaska         12.396667          -14.730000
2         Arizona          1.500000           -3.500000
3        Arkansas          6.845000          -26.920000
4      California         -6.490000           29.990000
..            ...               ...                 ...
48       Virginia          0.985556            5.320000
49     Washington          0.290000           15.710000
50  West Virginia         12.670000          -41.670000
51      Wisconsin          5.981538           -0.770000
52        Wyoming          6.790000          -46.290000

[53 rows x 3 columns]
(53, 3)


In [141]:
disp3 = polls[(polls['days_until'] < 90) & (polls['election'] == 'president') & (polls['year'] == 2012)]
disp3 = disp3.groupby(['state'])
disp3 = disp3[['margin_diff', 'actual_margin']].mean().reset_index()
disp3.columns = ['state', '2012_margin_diff', '2012_actual_margin']
show(disp2)

            state  2016_margin_diff  2016_actual_margin
0         Alabama          8.220000          -27.720000
1          Alaska         12.396667          -14.730000
2         Arizona          1.500000           -3.500000
3        Arkansas          6.845000          -26.920000
4      California         -6.490000           29.990000
..            ...               ...                 ...
48       Virginia          0.985556            5.320000
49     Washington          0.290000           15.710000
50  West Virginia         12.670000          -41.670000
51      Wisconsin          5.981538           -0.770000
52        Wyoming          6.790000          -46.290000

[53 rows x 3 columns]
(53, 3)


In [142]:
disp = disp.merge(disp2, how='left').merge(disp3, how='left') #.fillna(-20)
show(disp)

            state  polled_margin  actual_margin  2016_margin_diff  \
0         Alabama     -18.139388     -25.461653          8.220000   
1          Alaska      -8.147500     -10.061191         12.396667   
2         Arizona       4.312013       0.308710          1.500000   
3        Arkansas     -18.676364     -27.620668          6.845000   
4      California      27.367593      29.163223         -6.490000   
..            ...            ...            ...               ...   
45       Virginia      13.350536      10.114014          0.985556   
46     Washington      24.371739      19.203323          0.290000   
47  West Virginia     -31.196444     -38.935031         12.670000   
48      Wisconsin       8.369448       0.627100          5.981538   
49        Wyoming     -32.987442     -43.111923          6.790000   

    2016_actual_margin  2012_margin_diff  2012_actual_margin  
0           -27.720000               nan                 nan  
1           -14.730000               nan     

In [143]:
results = run_cv_model(disp[['polled_margin', '2012_margin_diff', '2016_margin_diff', '2016_actual_margin']], None, disp['actual_margin'].values, runRidge, {}, rmse)

Started model fold 1/5
[2021-03-11 23:34:29.938374] Fit Ridge


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
disp['pred_margin'] = results['train']
disp['pred_actual_diff'] = disp['pred_margin'] - disp['actual_margin']
disp['pred_actual_diff_abs'] = np.abs(disp['pred_margin'] - disp['actual_margin'])
show(disp.sort_values('pred_actual_diff_abs', ascending=False))

In [ ]:
disp[disp['state'] == 'Florida']

In [ ]:
disp[disp['state'] == 'Georgia']